<a href="https://colab.research.google.com/github/bill7845/colab_repo/blob/master/dacon_%EC%B2%9C%EC%B2%B4%EC%9C%A0%ED%98%95%EB%B6%84%EB%A5%98/%EC%B2%9C%EC%B2%B4%EC%9C%A0%ED%98%95%EB%B6%84%EB%A5%98_NOTE_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import missingno as msno
import scipy as sp

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split,GridSearchCV,KFold,cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score,log_loss
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier,AdaBoostClassifier,VotingClassifier,BaggingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

from sklearn.decomposition import PCA
from sklearn.decomposition import KernelPCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_curve
from yellowbrick.classifier import ROCAUC
from sklearn.metrics import classification_report

from xgboost import XGBClassifier
import xgboost as xgb
import lightgbm as lgb
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV

sns.set()

%matplotlib inline
%precision 3

'%.3f'

In [0]:
# train = pd.read_csv('C:/Users/KIHyuk/Desktop/dacon_data/Data_천체유형분류/train.csv',index_col=0)
# test = pd.read_csv('C:/Users/KIHyuk/Desktop/dacon_data/Data_천체유형분류/test.csv',index_col=0)
# sample_submission = pd.read_csv('C:/Users/KIHyuk/Desktop/dacon_data/Data_천체유형분류/sample_submission.csv',index_col=0)

train = pd.read_csv('/content/drive/My Drive/Colab Notebooks/dacon_천체유형분류/train.csv',index_col=0)
test = pd.read_csv('/content/drive/My Drive/Colab Notebooks/dacon_천체유형분류/test.csv',index_col=0)
sample_submission = pd.read_csv('/content/drive/My Drive/Colab Notebooks/dacon_천체유형분류/sample_submission.csv',index_col=0)

In [0]:
column_number = {}
for i, column in enumerate(sample_submission.columns):
    column_number[column] = i
    
def to_number(x, dic):
    return dic[x]

train['type_num'] = train['type'].apply(lambda x: to_number(x, column_number))

train = train.drop(['fiberID'],axis=1)
test = test.drop(['fiberID'],axis=1)

train_x,test_x,train_y,test_y = train_test_split(train.drop(columns=['type', 'type_num'], axis=1),train['type_num'],random_state=2,stratify=train['type_num'])

res_train_x = train.drop(columns=['type', 'type_num'], axis=1)
res_train_y = train['type_num']
res_test_x = test

In [0]:
## 차원축소

# scaling
scaler = StandardScaler()

sc_train_x = scaler.fit_transform(train_x)
sc_test_x = scaler.fit_transform(test_x)

# pca
pca = PCA(n_components=5,random_state=2)
pca_train_x = pca.fit_transform(sc_train_x)
pca_test_x = pca.transform(sc_test_x) ## test데이터에는 transform

In [0]:
# print('eigen_value :', pca.explained_variance_)
print('explained variance ratio :', pca.explained_variance_ratio_) # 주성분별 분산 설명 비율 (*설명비율)
print('선택한 차원(픽셀) 수 :', pca.n_components_) # 95%의 분산 설명을 위해 몇개의 차원을 선택했는가

explained variance ratio : [0.383 0.14  0.079 0.069 0.058]
선택한 차원(픽셀) 수 : 5


In [0]:
############################################################################
############ Random Forest
############################################################################
estimators = 300
np.random.seed(2)
RF_model = RandomForestClassifier(n_estimators = estimators)
RF_model.fit(train_x, train_y)
RF_prediction_proba = RF_model.predict_proba(test_x)
RF_prediction = RF_model.predict(test_x)

In [0]:
# 100
print("logloss : ",log_loss(test_y,RF_prediction_proba))

In [0]:
target_name = list(train['type'].unique())
print(classification_report(test_y,RF_prediction,target_names=list(train.type.unique())))

                     precision    recall  f1-score   support

                QSO       0.46      0.43      0.45       540
     STAR_RED_DWARF       0.96      0.96      0.96      1627
   SERENDIPITY_BLUE       0.60      0.39      0.47       125
           STAR_BHB       0.79      0.84      0.82       640
      STAR_CATY_VAR       0.90      0.97      0.94      3655
SERENDIPITY_DISTANT       0.85      0.90      0.87      3375
             GALAXY       0.97      0.98      0.97      9337
   SPECTROPHOTO_STD       0.55      0.44      0.49      1163
         REDDEN_STD       0.89      0.87      0.88     12420
            ROSAT_D       0.84      0.97      0.90        32
   STAR_WHITE_DWARF       0.97      0.98      0.97      3438
    SERENDIPITY_RED       0.62      0.29      0.39      1645
        STAR_CARBON       1.00      0.33      0.50         3
  SERENDIPITY_FIRST       0.76      0.60      0.67      1783
   STAR_BROWN_DWARF       0.82      0.94      0.88       814
     STAR_SUB_DWARF    

In [0]:
############################################################################
############ Random Forest _ PCA
############################################################################
estimators = 300
np.random.seed(2)
RF_model = RandomForestClassifier(n_estimators = estimators)
RF_model.fit(pca_train_x, train_y)
RF_prediction_pca_proba = RF_model.predict_proba(pca_test_x)
RF_prediction_pca = RF_model.predict(pca_test_x )

In [0]:
# 100
print("logloss : ",log_loss(test_y,RF_prediction_pca_proba))

logloss :  2.4404172692424253


In [0]:
print(classification_report(test_y,RF_prediction_pca,target_names=target_name))

                     precision    recall  f1-score   support

                QSO       0.00      0.00      0.00       540
     STAR_RED_DWARF       0.73      0.13      0.21      1627
   SERENDIPITY_BLUE       0.02      0.02      0.02       125
           STAR_BHB       0.08      0.30      0.13       640
      STAR_CATY_VAR       0.00      0.00      0.00      3655
SERENDIPITY_DISTANT       0.15      0.57      0.24      3375
             GALAXY       0.44      0.43      0.44      9337
   SPECTROPHOTO_STD       0.02      0.00      0.00      1163
         REDDEN_STD       0.52      0.74      0.61     12420
            ROSAT_D       0.06      0.25      0.09        32
   STAR_WHITE_DWARF       0.61      0.35      0.44      3438
    SERENDIPITY_RED       0.09      0.11      0.10      1645
        STAR_CARBON       0.00      0.00      0.00         3
  SERENDIPITY_FIRST       0.35      0.44      0.39      1783
   STAR_BROWN_DWARF       0.25      0.00      0.00       814
     STAR_SUB_DWARF    

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [0]:
submission_RF = pd.DataFrame(data=RF_prediction, columns=sample_submission.columns, index=test_x.index)
check_RF_accuracy = pd.DataFrame(data=RF_prediction, columns=range(0,19), index=test_x.index)
submission_RF.to_csv('submission_RF.csv', index=True)

In [0]:
############################################################################
############ Light GBM
############################################################################

param = {'num_leaves' :500,
        #  'min_data_in_leaf' : 50,
         'learning_rate' : 0.03,
         'max_depth' : 12,
         'objective' : 'softmax',
         'metric' : 'softmax',
         'num_class' : 19,
         'feature_fraction' : 0.7,
         'random_state' : 2}

# estimators = 100
np.random.seed(2)

train_data = lgb.Dataset(train_x, label=train_y)
num_round=1300

clf = lgb.train(param,train_data, num_round, valid_sets=[train_data], verbose_eval=100, early_stopping_rounds=200)
# LGBM_model.fit(train_x,train_y,param,early_stopping_rounds=200)
LGBM_prediction = clf.predict(test_x, num_iteration=clf.best_iteration)

Training until validation scores don't improve for 200 rounds.
[100]	training's multi_logloss: 0.320734
[200]	training's multi_logloss: 0.199374
[300]	training's multi_logloss: 0.163192
[400]	training's multi_logloss: 0.137532
[500]	training's multi_logloss: 0.11539
[600]	training's multi_logloss: 0.0958469
[700]	training's multi_logloss: 0.0795726
[800]	training's multi_logloss: 0.0661341
[900]	training's multi_logloss: 0.05563
[1000]	training's multi_logloss: 0.0466759
[1100]	training's multi_logloss: 0.0391969
[1200]	training's multi_logloss: 0.0329427
[1300]	training's multi_logloss: 0.0279569
Did not meet early stopping. Best iteration is:
[1300]	training's multi_logloss: 0.0279569


In [0]:
print("logloss : ",log_loss(test_y,LGBM_prediction))

logloss :  0.4210047425412792


In [0]:
submission_simpleLGBM = pd.DataFrame(data=LGBM_prediction, columns=sample_submission.columns, index=test_x.index)
check_LGBM_accuracy = pd.DataFrame(data=LGBM_prediction, columns=range(0,19), index=test_x.index)
submission_simpleLGBM.to_csv('submission_simpleLGBM.csv', index=True)

In [0]:
# ############################################################################
# ############ Logistic Regression
# ############################################################################

np.random.seed(2)
LR_model = LogisticRegression(random_state=2,multi_class='ovr',n_jobs=-1).fit(train_x,train_y)
LR_prediction = LR_model.predict_proba(test_x)

In [0]:
print("logloss : ",log_loss(test_y,LR_prediction))

logloss :  1.7145152146968106


In [0]:
submission_LR = pd.DataFrame(data=LR_prediction, columns=sample_submission.columns, index=test_x.index)
check_LR_accuracy = pd.DataFrame(data=LR_prediction, columns=range(0,19), index=test_x.index)
submission_LR.to_csv('submission_LR.csv', index=True)

In [0]:
# ############################################################################
# ############ SVM
# ############################################################################
SVM_model = svm.SVC(kernel='rbf',random_state=2,probability=True,gamma=0.3,C=1.0)
SVM_model.fit(train_x,train_y)
SVM_prediction = SVM_model.predict_proba(test_x)

In [0]:
submission_SVM = pd.DataFrame(data=SVM_prediction, columns=sample_submission.columns, index=test_x.index)
check_SVM_accuracy = pd.DataFrame(data=SVM_prediction, columns=range(0,19), index=test_x.index)
submission_SVM.to_csv('submission_SVM.csv', index=True)

In [0]:
################## checking################################
a = submission_RF.iloc[:,:].idxmax(axis=1)
# b = submission_simpleLGBM.iloc[:,:].idxmax(axis=1)
# c = submission_LR.iloc[:,:].idxmax(axis=1)
# d = submission_voting.iloc[:,:].idxmax(axis=1)


c_r = check_RF_accuracy.iloc[:,:].idxmax(axis=1)
# c_l = check_LGBM_accuracy.iloc[:,:].idxmax(axis=1)
# c_LR = check_LR_accuracy.iloc[:,:].idxmax(axis=1)
# c_voting = check_voting_accuracy.iloc[:,:].idxmax(axis=1)



# num=0
# for i in range(len(a)):
#   if a.iloc[i] == b.iloc[i]:
#     num = num + 1

# print(num)

def check_accuracy(check,test_y):
  tmp_num = 0
  for i,x in enumerate(check):
    if x == test_y.iloc[i]:
      tmp_num += 1
  print("전체 %d중 예측 일치 : %d " %(len(check),tmp_num))
  print("%f 예측" %(tmp_num/len(check)))

print(" #############RandomForest###############")
check_accuracy(c_r,test_y)
print(" ########################################")
print(" #############LightGBM###############")
# check_accuracy(c_l,test_y)
print(" ########################################")
print(" #############Logistic Regression###############")
# check_accuracy(c_LR,test_y)
print(" ########################################")
print(" #############Voting###############")
# check_accuracy(c_voting,test_y)

 #############RandomForest###############
전체 49998중 예측 일치 : 17412 
0.348254 예측
 ########################################
 #############LightGBM###############
 ########################################
 #############Logistic Regression###############
 ########################################
 #############Voting###############


In [0]:
############################################################################
# Voting
############################################################################
clf_1 = LogisticRegression(random_state=2, multi_class='auto', n_jobs=-1)
clf_2 = RandomForestClassifier(n_estimators=200,random_state=2) 
clf_3 = lgb.LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.03, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=500, n_jobs=-1, num_leaves=500, objective='softmax',
               random_state=2, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

voting_clf = VotingClassifier(estimators=[('svm',SVM_model),('rf',clf_2),('lgbm',clf_3)],voting='soft')
voting_clf.fit(res_train_x,res_train_y)
voting_prediction = voting_clf.predict_proba(res_test_x)

#### 최근우 읽기


In [0]:
submission_voting = pd.DataFrame(data=voting_prediction, columns=sample_submission.columns, index=res_test_x.index)
check_voting_accuracy = pd.DataFrame(data=voting_prediction, columns=range(0,19), index=res_test_x.index)
submission_voting.to_csv('submission_voting.csv', index=True)

NameError: ignored